In [46]:
#!/usr/bin/env python3
# coding: utf-8

# Gibbs-Sampling procedure to compute the Probability Matrix of a Discrete-Time Markov
# Chain whose states are the d-dimensional cartesian product of the form 
# {0,1,...n-1} x {0,1,...n-1} x ... X {0,1,...n-1} (i.e. d-many products)
# 
# The target stationary distribution is expressed over the n**d many states 
#
# Written by Prof. R.S. Sreenivas for
# IE531: Algorithms for Data Analytics
#

import sys
import argparse
import random
import numpy as np 
import time
import math
import matplotlib.pyplot as plt
import itertools as it


In [84]:

# need this to keep the matrix print statements to 4 decimal places
np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})

# This function computes a random n-dimensional probability vector (whose entries sum to 1)
def generate_a_random_probability_vector(n) :
    random_darts = np.random.uniform(0, 1, n-1)
    sort_random_darts = sorted(random_darts)
    y = [sort_random_darts[i] if i == 0 else (sort_random_darts[i] - sort_random_darts[i-1]) for i in range(0, n-1)]
    y.extend([1 - sort_random_darts[n-2]])
    return y

# Two d-tuples x and y are Gibbs-Neighbors if they are identical, or they differ in value at just
# one coordinate
def check_if_these_states_are_gibbs_neighbors(x, y) :
    # x and y are dim-tuples -- we will assume they are not equal
    # count the number of coordinates where they differ
    gibbs_neighbours_difference = [0 if a == b else 1 for a, b in zip(x, y)]
    if(sum(gibbs_neighbours_difference) == 1):
        gibbs_neighbours = True
    else:
        gibbs_neighbours = False
    
    return gibbs_neighbours

# Given two Gibbs-Neighbors -- that are not identical -- find the coordinate where they differ in value
# this is the "free-coordinate" for this pair
def free_coordinates_of_gibbs_neighbors(x, y) :
    # we assume x and y are gibbs neighbors, then the must agree on at least (dim-1)-many coordinates
    # or, they will disagree on just one of the (dim)-many coordinates... we have to figure out which 
    # coordinate/axes is free
    free_index = [i for i, (a, b) in enumerate(zip(x, y)) if a != b]
    return free_index[0]

# x in a dim-tuple (i.e. if dim = 2, it is a 2-tuple; if dim = 4, it is a 4-tuple) state of the Gibbs MC
# each of the dim-many variables in the dim-tuple take on values over range(n)... this function returns 
# the lexicographic_index (i.e. dictionary-index) of the state x
def get_lexicographic_index(x, n, dim) :
    all_possible_lexicographic_combo = list(it.product(range(n), repeat = dim))
    number = all_possible_lexicographic_combo.index(x)
    return number

# This is an implementaton of the Gibbs-Sampling procedure
# The MC has n**dim many states; the target stationary distribution is pi
# The third_variable_is when set to True, prints the various items involved in the procedure
# (not advisable to print for large MCs)
def create_gibbs_MC(n, dim, pi, do_want_to_print) :
    if (do_want_to_print) :
        print ("Generating the Probability Matrix using Gibbs-Sampling")
        print ("Target Stationary Distribution:")
        for x in it.product(range(n), repeat = dim) :
            number = get_lexicographic_index(x, n, dim)
            print ("\u03C0", x, " = \u03C0(", number , ") = ", pi[number])
    
    # the probability matrix will be (n**dim) x (n**dim) 
    probability_matrix = [[0 for x in range(n**dim)] for y in range(n**dim)]
    
    # the state of the MC is a dim-tuple (i.e. if dim = 2, it is a 2-tuple; if dim = 4, it is a 4-tuple)
    # got this from https://stackoverflow.com/questions/7186518/function-with-varying-number-of-for-loops-python
    for x in it.product(range(n), repeat = dim) :
        # x is a dim-tuple where each variable ranges over 0,1,...,n-1
        row_sum = 0
        for y in it.product(range(n), repeat = dim) :
            
            gibbs_neighbours = check_if_these_states_are_gibbs_neighbors(x, y)
            if gibbs_neighbours == True:
                denominator = 0
                free_coordinates_index = free_coordinates_of_gibbs_neighbors(x, y)
                for i in range(n):
                    convert_tuple_to_list = list(y)
                    convert_tuple_to_list[free_coordinates_index] = i
                    denominator += pi[get_lexicographic_index(tuple(convert_tuple_to_list), n, dim)]
                
                stochastic_value = (pi[get_lexicographic_index(y, n, dim)] / (denominator * dim))
                probability_matrix[get_lexicographic_index(x, n, dim)][get_lexicographic_index(y, n, dim)] = stochastic_value
                row_sum += stochastic_value 
        probability_matrix[get_lexicographic_index(x, n, dim)][get_lexicographic_index(x, n, dim)] = (1 - row_sum)
    return probability_matrix

# Trial 1... States: {(0,0), (0,1), (1,0), (1,1)} (i.e. 4 states)
n = 2
dim = 2
a = generate_a_random_probability_vector(n**dim)
print("(Random) Target Stationary Distribution\n", a)
p = create_gibbs_MC(n, dim, a, True) 
print ("Probability Matrix:")
print (np.matrix(p))
print ("Does the Probability Matrix have the desired Stationary Distribution?", np.allclose(np.matrix(a), np.matrix(a)* np.matrix(p)))



(Random) Target Stationary Distribution
 [0.34901442654895465, 0.09004969832438348, 0.03742685607669993, 0.5235090190499619]
Generating the Probability Matrix using Gibbs-Sampling
Target Stationary Distribution:
π (0, 0)  = π( 0 ) =  0.34901442654895465
π (0, 1)  = π( 1 ) =  0.09004969832438348
π (1, 0)  = π( 2 ) =  0.03742685607669993
π (1, 1)  = π( 3 ) =  0.5235090190499619
Probability Matrix:
[[0.8490 0.1025 0.0484 0.0000]
 [0.3975 0.1759 0.0000 0.4266]
 [0.4516 0.0000 0.0818 0.4666]
 [0.0000 0.0734 0.0334 0.8933]]
Does the Probability Matrix have the desired Stationary Distribution? True


In [85]:
# Trial 2... States{(0,0), (0,1),.. (0,9), (1,0), (1,1), ... (9.9)} (i.e. 100 states)
n = 10
dim = 2
a = generate_a_random_probability_vector(n**dim)
p = create_gibbs_MC(n, dim, a, False) 
print ("Does the Probability Matrix have the desired Stationary Distribution?", np.allclose(np.matrix(a), np.matrix(a)* np.matrix(p)))



Does the Probability Matrix have the desired Stationary Distribution? True


In [86]:
# Trial 3... 1000 states 
n = 10
dim = 3
t1 = time.time()
a = generate_a_random_probability_vector(n**dim)
p = create_gibbs_MC(n, dim, a, False) 
t2 = time.time()
hours, rem = divmod(t2-t1, 3600)
minutes, seconds = divmod(rem, 60)
print ("It took ", hours, "hours, ", minutes, "minutes, ", seconds, "seconds to finish this task")
print ("Does the Probability Matrix have the desired Stationary Distribution?", np.allclose(np.matrix(a), np.matrix(a)* np.matrix(p)))

It took  0.0 hours,  0.0 minutes,  22.744892120361328 seconds to finish this task
Does the Probability Matrix have the desired Stationary Distribution? True


In [87]:
# Trial 4... 10000 states 
n = 10
dim = 4
t1 = time.time()
a = generate_a_random_probability_vector(n**dim)
p = create_gibbs_MC(n, dim, a, False) 
t2 = time.time()
hours, rem = divmod(t2-t1, 3600)
minutes, seconds = divmod(rem, 60)
print ("It took ", hours, "hours, ", minutes, "minutes, ", seconds, "seconds to finish this task")
print ("Does the Probability Matrix have the desired Stationary Distribution?", np.allclose(np.matrix(a), np.matrix(a)* np.matrix(p)))


It took  2.0 hours,  44.0 minutes,  55.38531804084778 seconds to finish this task
Does the Probability Matrix have the desired Stationary Distribution? True


In [5]:
# This function computes a random n-dimensional probability vector (whose entries sum to 1)
def generate_a_random_probability_vector(n) :
    random_darts = np.random.uniform(0, 1, n-1)
    y = [random_darts[i] - random_darts[i-1] for i in range(1, n-1)]
    y.extend([1 - random_darts[n-1]])
    #*WRITE THIS PART **
    return y

In [4]:
x = (0, 0, 0, 0)
y = (0, 0, 0, 1)

# x and y are dim-tuples -- we will assume they are not equal
# count the number of coordinates where they differ
gibbs_neighbours_difference = [0 if a == b else 1 for a, b in zip(x, y)]
if(sum(gibbs_neighbours_difference) == 1):
    gibbs_neighbours = True
else:
    gibbs_neighbours = False

#return gibbs_neighbours


In [5]:
gibbs_neighbours

True

In [33]:
print(random_darts)
print(sort_random_darts)
print(y)

[0.93263478 0.5083562  0.87484274 0.79454532]
[0.5083561988339086, 0.7945453184318008, 0.8748427391364527, 0.9326347837999245]
[0.5083561988339086, 0.28618911959789217, 0.08029742070465196, 0.057792044663471764, 0.06736521620007552]


In [8]:
y

(0, 0, 0, 1)

In [17]:
free_index = [i for i, (a, b) in enumerate(zip(x, y)) if a != b]
(free_index)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [18]:
type(y)

list

In [10]:
for i, (a, b) in enumerate(zip(x, y)):
    print (i, a, b)

0 0 0
1 0 0
2 0 0
3 0 1


In [39]:
for x in it.product(range(3), repeat = 2):
    print(x)

(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)
(2, 0)
(2, 1)
(2, 2)


In [41]:
for x in it.product(range(1a ), range(3)):
    print(x)

(0, 0)
(0, 1)
(0, 2)


In [37]:
n = 3
dim = 1
x = (0, 1)
a = list(it.product(range(n), repeat = dim))
b = a.index(x)
print(a)
b


ValueError: (0, 1) is not in list

In [29]:
n = 3
dim = 2
probability_matrix = [[0 for x in range(n**dim)] for y in range(n**dim)]

In [35]:
probability_matrix


[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [45]:
x = (0, 1)
y = range(3)
z = list(x)
for i in range(3):
    z[0] = i
    print(tuple(z))


(0, 1)
(1, 1)
(2, 1)


In [65]:
n = 2
dim = 2
a = generate_a_random_probability_vector(n**dim)
pi = a
print(pi)
print(pi[0]+pi[1])

[0.2240543291693049, 0.012717993298108188, 0.4747383598412024, 0.28848931769138453]
0.2367723224674131


In [66]:
for x in it.product(range(n), repeat = dim) :
    number = get_lexicographic_index(x, n, dim)
    print ("\u03C0", x, " = \u03C0(", number , ") = ", pi[number])

π (0, 0)  = π( 0 ) =  0.2240543291693049
π (0, 1)  = π( 1 ) =  0.012717993298108188
π (1, 0)  = π( 2 ) =  0.4747383598412024
π (1, 1)  = π( 3 ) =  0.28848931769138453


In [67]:
probability_matrix = [[0 for x in range(n**dim)] for y in range(n**dim)]
probability_matrix

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [68]:
x = (0, 0)
y = (0, 1)
gibbs_neighbours = check_if_these_states_are_gibbs_neighbors(x, y)
gibbs_neighbours


True

In [69]:
denominator = 0
free_coordinates_index = free_coordinates_of_gibbs_neighbors(x, y)
for i in range(n):
    convert_tuple_to_list = list(y)
    convert_tuple_to_list[free_coordinates_index] = i
    denominator += pi[get_lexicographic_index(tuple(convert_tuple_to_list), n, dim)]
print(denominator)


0.2367723224674131


In [70]:
stochastic_value = (pi[get_lexicographic_index(y, n, dim)] / (denominator * dim))
probability_matrix.append(stochastic_value)

In [74]:
probability_matrix[get_lexicographic_index(x, n, dim)][get_lexicographic_index(x, n, dim)] = stochastic_value

In [73]:
get_lexicographic_index(x, n, dim)

0

In [75]:
probability_matrix

[[0.026857010071053726, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 0.026857010071053726]